In [8]:
import pandas as pd 
import string
import numpy as np

# note: this is the dataframe acquired from webscraping and then merging with the metascore df
df = pd.read_csv('fp2.csv')

df

,Unnamed: 0,Title,Year,Publisher,Genre,Platform,Metascore,Avg_Userscore,No_Players,Developer:,Discounted Price,Genre:,Number of Ratings,Percentage of Positive Reviews,Price,Publisher:,Release Date:,Title:,Franchise:
0,0,The Legend of Zelda: Ocarina of Time,1998,Nintendo,Action Adventure;Fantasy,Nintendo64,99,9.1,1 Player,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Skateboarding,PlayStation,98,7.4,1-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Tony Hawk's Pro Skater 2,2000,Treyarch,Sports;Alternative;Skateboarding,Dreamcast,97,6.2,not specified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Tony Hawk's Pro Skater 2,2001,VicariousVisions,Sports;Alternative;Skateboarding,GameBoyAdvance,95,7.0,not specified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Tony Hawk's Pro Skater 2,2000,NeversoftEntertainment,Sports;Alternative;Individual;Skateboarding;Sk...,PC,91,8.5,1-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,5847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Blueside'],NaN,"['Action', 'Massively Multiplayer', 'RPG', 'St...",1399.0,73%,$9.99,['Gameforge 4D GmbH\t'],['Nov182019'],['KingdomUnderFire2'],NaN
5848,5848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Eidos-Montréal', 'Feral Interactive (Mac)']",NaN,"['Action', 'Adventure']",11296.0,72%,$19.99,"['Square Enix', 'Feral Interactive (Mac)']",['Feb272014'],['Thief'],['Thief']
5849,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['LucasArts'],NaN,['Action'],3805.0,69%,$19.99,"['LucasArts', 'Lucasfilm', 'Disney']",['Nov32009'],['STARWARSTheForceUnleashedUltimateSithEdition'],['Star Wars']
5850,5850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Cyanide Studios', 'Cyanide Studio']",$8.99,"['Sports', 'Strategy', 'Violent', 'Gore']",0.0,No Rating,$29.99,['Focus Home Interactive'],['Sep52017'],['BloodBowl2LegendaryEdition'],['Warhammer']


In [9]:
def clean(col1, col2):
    df[col1] = df[col2].astype(str) + df[col1].astype(str)
    df[col1] = [w.replace('nan', '').strip('[]\'').replace("'", '').replace(', ', ';') for w in df[col1]]

for c1, c2 in zip(['Title', 'Year', 'Publisher', 'Genre'], ['Title:', 'Release Date:', 'Publisher:', 'Genre:']):
    clean(c1, c2)
    
df['Year'] = [y[-4:] if len(y) > 4 else y for y in df['Year']]
df['No_Players'] = [str(w).strip(string.ascii_letters+' ') if isinstance(w, str) else w for w in df['No_Players']]
df['No_Players'] = [entry.split('-')[1] if isinstance(entry, str) and '-' in entry else entry for entry in df['No_Players']]        
df = df[df['Percentage of Positive Reviews'].notna()]
df['Franchise:'] = [w.strip('[]\'') if isinstance(w, str) else w for w in df['Franchise:']]
df['Developer:'] = [w.strip('[]\'') if isinstance(w, str) else w for w in df['Developer:']]
df['Genre'] = [w.replace(']', ';') for w in df['Genre']]
df = df.drop(['Title:', 'Release Date:', 'Publisher:', 'Genre:', 'Publisher', 'Developer:', 'Discounted Price', 'Unnamed: 0'], axis=1)
df = df.dropna(thresh=6)

def x(s):
    if s != 0:
        return 1
    else:
        return 0
    
def changeF2PtoZero(price):
    if type(price) == str:
        price = price.lower()
        if "free" in price:
            return 0
        else:
            return float(price)
    return price

df['Franchise:'] = df['Franchise:'].fillna(0)
df['Franchise:'] = df['Franchise:'].map(x)
df['Percentage of Positive Reviews'] = [w.strip('%') if isinstance(w, str) else w for w in df['Percentage of Positive Reviews']]
df['Price'] = [w.strip('$') if isinstance(w, str) else w for w in df['Price']]
df['Price'] = df['Price'].map(changeF2PtoZero)
df = df[df['Percentage of Positive Reviews'] != 'No Rating']

# remove percentage and dollar sign, add genre columns

df['Platform'].fillna('', inplace=True)
df = df[(df["Platform"] == "") | (df["Platform"] == "PC")]
df.drop("Platform", axis = 1, inplace = True)


C:\Users\drewb\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\drewb\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\drewb\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [10]:
import re
def find_average(series):
    total = 0
    length = 0
    for e in series:
        try:
            val = str(e)
            loi = [s for s in val if s.isdigit()]
            if loi:
                total += int(''.join(loi))
                length += 1
        except ValueError:
            try:
                total += val
                length += 1
            except ValueError:
                print('error')
                print(type(e))
                print(e)
                
    return total/length
meta_avg = find_average(df['Metascore'])
user_avg = find_average(df['Avg_Userscore'])
player_avg = find_average(df['No_Players'])
price_avg = find_average(df['Price'])
print(price_avg)
print(df["Price"].mean())

df['Metascore'].fillna(round(meta_avg), inplace=True)
df['Avg_Userscore'].fillna(round(user_avg/10., 1), inplace=True)
df['No_Players'].fillna(round(player_avg), inplace=True)

def turnEmpyStringToAvgNumPlayers(no_players):
    if no_players == "":
        return round(player_avg)
    else:
        return no_players
    
def turnEmptyStringToAvgYear(year):
    if year == "":
        # returning the average year
        return 2016
    else:
        return int(year)
    
    

df["No_Players"] = df["No_Players"].map(turnEmpyStringToAvgNumPlayers)
df["Year"] = df["Year"].map(turnEmptyStringToAvgYear)

df['Price'].fillna(round(df["Price"].mean(),2), inplace = True)
df

2326.045861297539
23.290659955257507


,Title,Year,Genre,Metascore,Avg_Userscore,No_Players,Number of Ratings,Percentage of Positive Reviews,Price,Franchise:
17,GrandTheftAutoV]Grand Theft Auto V,2015,Action;Adventure;Modern;Action Adventure;Open-...,96,7.7,32,773837.0,79,29.99,1
54,MassEffect2]Mass Effect 2,2010,RPG;Role-Playing;Action RPG;Action RPG,94,8.8,11,11227.0,94,19.99,1
89,Portal2]Portal 2,2011,Action;Adventure;Action;Shooter;Shooter;First-...,95,9.0,11,150366.0,98,9.99,0
99,BioShockInfinite]BioShock Infinite,2013,Action;Action;Shooter;Shooter;First-Person;Sci...,94,8.6,11,69546.0,95,29.99,1
143,Celeste]Celeste,2018,Action;Adventure;Indie;Action;Platformer;2D,88,7.2,11,19317.0,97,19.99,0
...,...,...,...,...,...,...,...,...,...,...
5846,ZooTycoonUltimateAnimalCollection,2018,Simulation,85,7.9,11,598.0,57,19.99,0
5847,KingdomUnderFire2,2019,Action;Massively Multiplayer;RPG;Strategy,85,7.9,11,1399.0,73,9.99,0
5848,Thief,2014,Action;Adventure,85,7.9,11,11296.0,72,19.99,1
5849,STARWARSTheForceUnleashedUltimateSithEdition,2009,Action,85,7.9,11,3805.0,69,19.99,1


In [11]:
s = df['Genre']
s = [entry.split(';') for  entry in s.values]

In [12]:
def count_unique(series):
    
    unique_items = []
    
    for item in series:
        for i in range(len(item)):
            
            if item[i] not in unique_items:
                
                unique_items.append(item[i])
    unique_items = {item:i for i, item in enumerate(unique_items)}
    return unique_items

In [13]:
import numpy as np
def one_hot(series, unique_lst):
    encoded = np.zeros((len(series), len(unique_lst)))
    for i, genres in enumerate(series):
        encoded[i][[unique_lst[g] for g in genres]] = 1
        
    return encoded

In [14]:
unique = count_unique(s)
genres = one_hot(s, unique)
for g in unique.keys():
    if sum(genres[:, unique[g]]) >= 5:
        df[g] = genres[:, unique[g]]
df = df.drop('Genre', axis=1)

In [15]:
cleaned_df = df

In [16]:
cleaned_df.reset_index(inplace = True, drop = True)

In [17]:
cleaned_df = cleaned_df.rename(columns = {"Franchise:": "Franchise"})

In [18]:
cleaned_df

,Title,Year,Metascore,Avg_Userscore,No_Players,Number of Ratings,Percentage of Positive Reviews,Price,Franchise,Action,...,General,Free to Play,Simulation,Turn-Based,Racing,Sports,Massively Multiplayer,Casual,Early Access,Design & Illustration
0,GrandTheftAutoV]Grand Theft Auto V,2015,96,7.7,32,773837.0,79,29.99,1,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MassEffect2]Mass Effect 2,2010,94,8.8,11,11227.0,94,19.99,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Portal2]Portal 2,2011,95,9.0,11,150366.0,98,9.99,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BioShockInfinite]BioShock Infinite,2013,94,8.6,11,69546.0,95,29.99,1,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Celeste]Celeste,2018,88,7.2,11,19317.0,97,19.99,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,ZooTycoonUltimateAnimalCollection,2018,85,7.9,11,598.0,57,19.99,0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902,KingdomUnderFire2,2019,85,7.9,11,1399.0,73,9.99,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
903,Thief,2014,85,7.9,11,11296.0,72,19.99,1,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
904,STARWARSTheForceUnleashedUltimateSithEdition,2009,85,7.9,11,3805.0,69,19.99,1,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [341]:
def check_num_ones(column):
    num_ones = 0
    for i in range(len(cleaned_df)):
        if cleaned_df[column].iloc[i] == 1:
            num_ones += 1

    print(column, num_ones)

In [342]:
# checking how many games are in each genre
genres = cleaned_df.columns[9:]
for column in genres:
    check_num_ones(column)

Action 462
Adventure 252
Modern 8
Action Adventure 14
RPG 226
Role-Playing 14
Action RPG 6
Shooter 18
First-Person 21
Sci-Fi 10
Arcade 17
Indie 323
Platformer 8
2D 8
Strategy 320
Real-Time 7
Historic 5
General 31
Free to Play 79
Simulation 269
Turn-Based 5
Racing 23
Sports 37
Massively Multiplayer 90
Casual 88
Early Access 92
Design & Illustration 5


In [343]:
cleaned_df.to_csv("final_df.csv")